In [1]:
# path to your train/test/meta folders
DATA_PATH = 'E:/Mein/Учеба/Audio/'

# names of valuable files/folders
train_meta_fname = 'train.csv'
test_meta_fname = 'sample_submission.csv'
train_data_folder = 'train'
test_data_folder = 'test'

In [6]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
import torchvision
from torchaudio import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

In [7]:
# set seeds
import random
import numpy as np

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [8]:
df_train = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_test = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [9]:
n_classes = df_train.label.nunique()
print(n_classes)
classes_dict = {cl:i for i,cl in enumerate(df_train.label.unique())}
df_train['label_encoded'] = df_train.label.map(classes_dict)
df_train.head()

41


,fname,label,label_encoded
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping,0
1,00d77b917e241afa06f1.wav,Squeak,1
2,17bb93b73b8e79234cb3.wav,Electric_piano,2
3,7d5c7a40a936136da55e.wav,Harmonica,3
4,17e0ee7565a33d6c2326.wav,Snare_drum,4


In [10]:
# https://github.com/lukemelas/EfficientNet-PyTorch
class BaseLineModel(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MelSpectrogram(sample_rate)
#         self.bn1 = nn.BatchNorm2d(1)
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=3, kernel_size=3, padding=1)
        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')
        # use it as features
#         for param in self.features.parameters():
#             param.requires_grad = False
            
        self.lin1 = nn.Linear(1000, 333)
        
        self.lin2 = nn.Linear(333, 111)
                
        self.lin3 = nn.Linear(111, n_classes)
        
    def forward(self, x):
        x = self.ms(x)
#         x = self.bn1(x)
                
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))
        
        x = self.features(x)

        x = x.view(x.shape[0], -1)
        x = F.relu(x)
        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [11]:
def sample_or_pad(waveform, wav_len=32000):
    m, n = waveform.shape
    if n < wav_len:
        padded_wav = torch.zeros(1, wav_len)
        padded_wav[:, :n] = waveform
        return padded_wav
    elif n > wav_len:
        offset = np.random.randint(0, n - wav_len)
        sampled_wav = waveform[:, offset:offset+wav_len]
        return sampled_wav
    else:
        return waveform
        
class EventDetectionDataset(Dataset):
    def __init__(self, data_path, x, y=None):
        self.x = x
        self.y = y
        self.data_path = data_path
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        path2wav = os.path.join(self.data_path, self.x[idx])
        waveform, sample_rate = torchaudio.load(path2wav, normalization=True)
        waveform = sample_or_pad(waveform)
        if self.y is not None:
            return waveform, self.y[idx]
        return waveform

In [17]:
X_train, X_val, y_train, y_val = train_test_split(df_train.fname.values, df_train.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
val_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )
test_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, test_data_folder), df_test.fname.values, None),
                        batch_size=41, shuffle=False
                )

In [13]:
def eval_model(model, eval_dataset):
    model.eval()
    forecast, true_labs = [], []
    with torch.no_grad():
        for wavs, labs in tqdm(eval_dataset):
            wavs, labs = wavs.cuda(), labs.detach().numpy()
            true_labs.append(labs)
            outputs = model.inference(wavs)
            
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labs = [x for sublist in true_labs for x in sublist]
    return f1_score(forecast, true_labs, average='macro')

In [14]:
criterion = nn.CrossEntropyLoss()
model = BaseLineModel()
model = model.cuda()
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\fanpi/.cache\torch\checkpoints\efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [18]:
n_epoch = 45
best_f1 = 0
for epoch in range(n_epoch):
    model.train()
    for wavs, labs in tqdm(train_loader):
        optimizer.zero_grad()
        wavs, labs = wavs.cuda(), labs.cuda()
        outputs = model(wavs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
#     if epoch % 10 == 0:
    f1 = eval_model(model, val_loader)
    f1_train = eval_model(model, train_loader)
    print(f'epoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), 'baseline_fulldiv.pt')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr



  0%|          | 0/111 [00:00<?, ?it/s]

  1%|          | 1/111 [00:00<01:43,  1.06it/s]

  2%|▏         | 2/111 [00:01<01:27,  1.24it/s]

  3%|▎         | 3/111 [00:01<01:16,  1.41it/s]

  4%|▎         | 4/111 [00:02<01:08,  1.57it/s]

  5%|▍         | 5/111 [00:02<01:02,  1.70it/s]

  5%|▌         | 6/111 [00:03<00:57,  1.82it/s]

  6%|▋         | 7/111 [00:03<00:55,  1.87it/s]

  7%|▋         | 8/111 [00:04<00:52,  1.95it/s]

  8%|▊         | 9/111 [00:04<00:50,  2.00it/s]

  9%|▉         | 10/111 [00:05<00:49,  2.04it/s]

 10%|▉         | 11/111 [00:05<00:49,  2.04it/s]

 11%|█         | 12/111 [00:06<00:47,  2.06it/s]

 12%|█▏        | 13/111 [00:06<00:47,  2.08it/s]

 13%|█▎        | 14/111 [00:07<00:46,  2.09it/s]

 14%|█▎        | 15/111 [00:07<00:45,  2.10it/s]

 14%|█▍        | 16/111 [00:08<00:45,  2.11it/s]

 15%|█▌        | 17/111 [00:08<00:52,  1.80it/s]

 16%|█▌        | 18/111 [00:10<01:25,  1.08it/s]

 17%|█▋        | 19/111 [00:12<01:58,  1.29s/it]

 18%|█▊        |

epoch: 0, f1_test: 0.09052991225777804, f1_train: 0.08111042573052735




  1%|          | 1/111 [00:01<03:24,  1.86s/it]

  2%|▏         | 2/111 [00:03<03:10,  1.75s/it]

  3%|▎         | 3/111 [00:05<03:13,  1.79s/it]

  4%|▎         | 4/111 [00:06<03:05,  1.73s/it]

  5%|▍         | 5/111 [00:08<03:02,  1.72s/it]

  5%|▌         | 6/111 [00:10<03:11,  1.83s/it]

  6%|▋         | 7/111 [00:12<03:03,  1.77s/it]

  7%|▋         | 8/111 [00:13<03:01,  1.77s/it]

  8%|▊         | 9/111 [00:15<02:47,  1.64s/it]

  9%|▉         | 10/111 [00:17<02:48,  1.67s/it]

 10%|▉         | 11/111 [00:18<02:43,  1.63s/it]

 11%|█         | 12/111 [00:20<02:49,  1.72s/it]

 12%|█▏        | 13/111 [00:22<02:48,  1.72s/it]

 13%|█▎        | 14/111 [00:24<02:50,  1.76s/it]

 14%|█▎        | 15/111 [00:25<02:47,  1.75s/it]

 14%|█▍        | 16/111 [00:27<02:37,  1.66s/it]

 15%|█▌        | 17/111 [00:29<02:48,  1.80s/it]

 16%|█▌        | 18/111 [00:30<02:41,  1.73s/it]

 17%|█▋        | 19/111 [00:32<02:31,  1.65s/it]

 18%|█▊        | 20/111 [00:34<02:30,  1.65s/it]

 19%|█▉

epoch: 1, f1_test: 0.2219402876963495, f1_train: 0.24852819627076264




  1%|          | 1/111 [00:01<03:37,  1.98s/it]

  2%|▏         | 2/111 [00:03<03:25,  1.89s/it]

  3%|▎         | 3/111 [00:05<03:34,  1.98s/it]

  4%|▎         | 4/111 [00:07<03:27,  1.94s/it]

  5%|▍         | 5/111 [00:09<03:20,  1.89s/it]

  5%|▌         | 6/111 [00:11<03:10,  1.81s/it]

  6%|▋         | 7/111 [00:12<03:03,  1.77s/it]

  7%|▋         | 8/111 [00:14<03:05,  1.80s/it]

  8%|▊         | 9/111 [00:15<02:49,  1.66s/it]

  9%|▉         | 10/111 [00:17<02:52,  1.71s/it]

 10%|▉         | 11/111 [00:19<02:54,  1.75s/it]

 11%|█         | 12/111 [00:21<02:44,  1.66s/it]

 12%|█▏        | 13/111 [00:22<02:46,  1.69s/it]

 13%|█▎        | 14/111 [00:24<02:55,  1.81s/it]

 14%|█▎        | 15/111 [00:26<02:47,  1.74s/it]

 14%|█▍        | 16/111 [00:28<02:53,  1.83s/it]

 15%|█▌        | 17/111 [00:30<02:47,  1.78s/it]

 16%|█▌        | 18/111 [00:32<02:45,  1.78s/it]

 17%|█▋        | 19/111 [00:34<02:51,  1.86s/it]

 18%|█▊        | 20/111 [00:35<02:48,  1.85s/it]

 19%|█▉

...

epoch: 42, f1_test: 0.49584099454571223, f1_train: 0.6619332269447072




  1%|          | 1/111 [00:02<04:26,  2.43s/it]

  2%|▏         | 2/111 [00:04<04:04,  2.24s/it]

  3%|▎         | 3/111 [00:06<03:59,  2.22s/it]

  4%|▎         | 4/111 [00:08<03:47,  2.13s/it]

  5%|▍         | 5/111 [00:11<04:14,  2.40s/it]

  5%|▌         | 6/111 [00:13<03:54,  2.23s/it]

  6%|▋         | 7/111 [00:15<03:42,  2.14s/it]

  7%|▋         | 8/111 [00:17<03:59,  2.33s/it]

  8%|▊         | 9/111 [00:19<03:36,  2.12s/it]

  9%|▉         | 10/111 [00:22<04:04,  2.42s/it]

 10%|▉         | 11/111 [00:24<03:58,  2.38s/it]

 11%|█         | 12/111 [00:26<03:38,  2.21s/it]

 12%|█▏        | 13/111 [00:28<03:30,  2.15s/it]

 13%|█▎        | 14/111 [00:30<03:21,  2.08s/it]

 14%|█▎        | 15/111 [00:32<03:22,  2.11s/it]

 14%|█▍        | 16/111 [00:34<03:17,  2.08s/it]

 15%|█▌        | 17/111 [00:36<03:12,  2.05s/it]

 16%|█▌        | 18/111 [00:38<03:00,  1.94s/it]

 17%|█▋        | 19/111 [00:40<02:55,  1.91s/it]

 18%|█▊        | 20/111 [00:50<06:31,  4.31s/it]

 19%|█▉

epoch: 43, f1_test: 0.4990265268618951, f1_train: 0.6788588566906




  1%|          | 1/111 [00:02<04:24,  2.41s/it]

  2%|▏         | 2/111 [00:04<04:01,  2.21s/it]

  3%|▎         | 3/111 [00:06<03:56,  2.19s/it]

  4%|▎         | 4/111 [00:09<04:14,  2.38s/it]

  5%|▍         | 5/111 [00:11<03:58,  2.25s/it]

  5%|▌         | 6/111 [00:12<03:42,  2.12s/it]

  6%|▋         | 7/111 [00:14<03:30,  2.03s/it]

  7%|▋         | 8/111 [00:16<03:24,  1.99s/it]

  8%|▊         | 9/111 [00:18<03:11,  1.88s/it]

  9%|▉         | 10/111 [00:20<03:13,  1.92s/it]

 10%|▉         | 11/111 [00:22<03:07,  1.88s/it]

 11%|█         | 12/111 [00:23<02:56,  1.79s/it]

 12%|█▏        | 13/111 [00:25<03:11,  1.95s/it]

 13%|█▎        | 14/111 [00:27<03:07,  1.93s/it]

 14%|█▎        | 15/111 [00:29<03:08,  1.96s/it]

 14%|█▍        | 16/111 [00:31<02:57,  1.87s/it]

 15%|█▌        | 17/111 [00:33<03:00,  1.92s/it]

 16%|█▌        | 18/111 [00:35<02:53,  1.86s/it]

 17%|█▋        | 19/111 [00:37<03:01,  1.98s/it]

 18%|█▊        | 20/111 [00:39<02:55,  1.92s/it]

 19%|█▉

epoch: 44, f1_test: 0.5202285481017163, f1_train: 0.7272943216978288


In [20]:
# make a model
model_name = 'baseline_fulldiv.pt'
model = BaseLineModel().cuda()
model.load_state_dict(torch.load(os.path.join(model_name)))
model.eval()
forecast = []
with torch.no_grad():
    for wavs in tqdm(test_loader):
        wavs = wavs.cuda()
        outputs = model.inference(wavs)
        outputs = outputs.detach().cpu().numpy().argmax(axis=1)
        forecast.append(outputs)
forecast = [x for sublist in forecast for x in sublist]
decoder = {classes_dict[cl]:cl for cl in classes_dict}
forecast = pd.Series(forecast).map(decoder)
df_test['label'] = forecast
df_test.to_csv(f'{model_name}.csv', index=None)

Loaded pretrained weights for efficientnet-b0




  0%|          | 0/93 [00:00<?, ?it/s]E:\Anaconda\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  1%|          | 1/93 [00:01<02:43,  1.78s/it]

  2%|▏         | 2/93 [00:03<02:30,  1.66s/it]

  3%|▎         | 3/93 [00:04<02:19,  1.55s/it]

  4%|▍         | 4/93 [00:06<02:18,  1.56s/it]

  5%|▌         | 5/93 [00:07<02:12,  1.51s/it]

  6%|▋         | 6/93 [00:09<02:15,  1.56s/it]

  8%|▊         | 7/93 [00:10<02:16,  1.58s/it]

  9%|▊         | 8/93 [00:12<02:16,  1.61s/it]

 10%|▉         | 9/93 [00:14<02:15,  1.62s/it]

 11%|█         | 10/93 [00:16<02:36,  1.88s/it]

 12%|█▏        | 11/93 [00:18<02:38,  1.93s/it]

 13%|█▎        | 12/93 [00:20<02:25,  1.80s/it]

 14%|█▍        | 13/93 [00:21<02:14,  1.68s/it]

 15%|█▌        | 14/93 [00:23<02:22,  1.81s/it]

 16%|█▌        | 15/93 [00:25<02:14,  1.72s/it]

 17%|█▋        | 16/93 [00:26<02:05,  1.62s/it]

 18%|█